# 페이퍼의 이름에 저널이 포함되어 있는 경우를 다시 나눠주는 튜닝작업

#### 튜닝 작업에는 기존에 저널 카테고리에 있던 저널 이름들을 일반화시켜주는 작업을 한 이후(journal_name_ls)에 이를 바탕으로 함

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
import re
from collections import Counter
import nltk

In [2]:
df = pd.read_csv('tuned_dataframe.csv')

In [3]:
df.tail()

,paper_name_infor,institution_infor,author_infor,journal_infor,dont_know_infor,page_infor,year_infor,field_infor,cited_infor,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
37242,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[기타자료],NaN,NaN,"Villalonga, Belen",Does Diversification Cause the “Diversificatio...,NaN,1999.0
37243,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Journal of Finance 59 : 479 ~ 506,","Villalonga, Belen",Diversification discount or premium? New Evide...,NaN,2004.0
37244,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Financial Management 33 : 5 ~ 27,","Villalonga, Belen",Does Diversification Cause the Diversification...,NaN,2004.0
37245,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"St. John’s Review 44 : 66 ~ 80,","Weston, J. F",The Nature and Significance of Conglomerate Firms,NaN,1970.0
37246,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,Journal of Finance 56 : 1667 ~ 1691,"Whited, Toni M",Is It Inefficient Investment That Causes the D...,NaN,2001.0


In [4]:
100 * df.isnull().sum() / len(df)

paper_name_infor      0.000000
institution_infor     0.000000
author_infor          0.000000
journal_infor         0.000000
dont_know_infor       0.000000
page_infor            0.000000
year_infor            0.000000
field_infor           0.000000
cited_infor           0.000000
char_refer            0.649717
degree_refer         99.747631
journal_refer        22.570945
name_refer           12.041238
paper_refer           0.649717
school_refer         99.747631
year_refer            0.649717
dtype: float64

# 일단 제목에 Journal 이 있으면 split 해준다.

In [5]:
%%time
for idx,val in enumerate(df['paper_refer']) : 
    if df['paper_refer'].notnull()[idx]:
        if 'Journal' in val : 
            df['journal_refer'][idx] = val[re.search('Journal',val).start():]
            df['paper_refer'][idx] = val[:re.search('Journal',val).start()]

/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


CPU times: user 1min 54s, sys: 33 ms, total: 1min 54s
Wall time: 1min 54s


In [6]:
[i for i in list(set(df['paper_refer']))[2:] if 'Journal' in i]

[]

저널 컬럼의 결측치 값이 2퍼센트정도 줄었다.

In [7]:
100 * df.isnull().sum() / len(df)

paper_name_infor      0.000000
institution_infor     0.000000
author_infor          0.000000
journal_infor         0.000000
dont_know_infor       0.000000
page_infor            0.000000
year_infor            0.000000
field_infor           0.000000
cited_infor           0.000000
char_refer            0.649717
degree_refer         99.747631
journal_refer        20.017720
name_refer           12.041238
paper_refer           0.649717
school_refer         99.747631
year_refer            0.649717
dtype: float64

# 다른 저널명들은 기존의 저널 컬럼에서 데이터를 가지고 와 이를 기반으로 찾아준다.

In [8]:
journal_name_ls = list(pd.read_csv('journal_name_ls.csv')['0'].values)

In [13]:
journal_name_ls.append('working paper')

In [14]:
for idx,val in enumerate(journal_name_ls) : 
    if val[-1] == ' ':
        journal_name_ls[idx] = val[:-1]

In [15]:
journal_name_ls = list(set(journal_name_ls))

In [16]:
journal_name_ls[:5]

['경제학연구',
 'Experimental Economics',
 'Journal ofMultinational Financial Management',
 'Financial Markets Institutions & Instruments',
 'Annals of Economics and Finance']

In [17]:
journal_name_ls = [i for i in journal_name_ls if len(i) >2]

In [18]:
split_ls = []
for idx,val in enumerate(df['paper_refer']):
    if df['paper_refer'].notnull()[idx]:
        for j in journal_name_ls:
            if j in val:
                split_ls.append((idx,j))
                break
            

In [19]:
split_ls[:5]

[(5, 'Jan'), (6, 'Infor'), (12, 'Risk'), (13, 'Risk'), (16, 'Risk')]

- 살펴본 결과, 한글일 경우 유의미할 가능성이 높았고, 영어일 경우 Infor , Focus와 같은 단어들은 좋지 않은 Threshold 일 가능성이 높았다.
- 이에 따라서, split_ls 에서 한글은 무조건적으로 집어넣고, 영어인 경우 길이가 10 를 넘어서는 것들만 취합하였다.
- 10으로 숫자를 크게 잡은 이유는 아닌 것을 맞다라고 하는 Type2 error 를 방지하는 것이 우선이라고 생각했기 때문이다.

In [20]:
new_split_ls = [split_ls[i] for i in range(len(split_ls)) if (len(split_ls[i][1]) > 10) or  (re.findall('[가-힣]',split_ls[i][1]))]
new_split_ls[:5]

[(22, 'Working Paper'),
 (33, 'Corporate Governance'),
 (89, 'Brookings-Wharton Papers on Financial Services'),
 (100, 'Federal Reserve Bank of Chicago'),
 (193, 'Economic Theory')]

In [21]:
%%time
for idx,val in enumerate(df['paper_refer']) : 
    if idx in [i[0] for i in new_split_ls] :
        try : 
            loc = [i[0] for i in new_split_ls].index(idx)
            df['journal_refer'][idx] = val[re.search(new_split_ls[loc][1],df['paper_refer'][idx]).start() : ]
            df['paper_refer'][idx] = val[ : re.search(new_split_ls[loc][1],df['paper_refer'][idx]).start() ]
        except : print(idx,loc)

/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


CPU times: user 1min 18s, sys: 3.68 ms, total: 1min 18s
Wall time: 1min 18s


In [22]:
100 * df.isnull().sum() / len(df)

paper_name_infor      0.000000
institution_infor     0.000000
author_infor          0.000000
journal_infor         0.000000
dont_know_infor       0.000000
page_infor            0.000000
year_infor            0.000000
field_infor           0.000000
cited_infor           0.000000
char_refer            0.649717
degree_refer         99.747631
journal_refer        18.731710
name_refer           12.041238
paper_refer           0.649717
school_refer         99.747631
year_refer            0.649717
dtype: float64

- journal refer 에 대한 결측치를 22.57 -> 20.01 -> 18.8 퍼센트로 줄였다.

In [23]:
df.tail()

,paper_name_infor,institution_infor,author_infor,journal_infor,dont_know_infor,page_infor,year_infor,field_infor,cited_infor,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
37242,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[기타자료],NaN,"Working paper, Harvard University,","Villalonga, Belen",Does Diversification Cause the “Diversificatio...,NaN,1999.0
37243,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Journal of Finance 59 : 479 ~ 506,","Villalonga, Belen",Diversification discount or premium? New Evide...,NaN,2004.0
37244,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Financial Management 33 : 5 ~ 27,","Villalonga, Belen",Does Diversification Cause the Diversification...,NaN,2004.0
37245,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"St. John’s Review 44 : 66 ~ 80,","Weston, J. F",The Nature and Significance of Conglomerate Firms,NaN,1970.0
37246,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,Journal of Finance 56 : 1667 ~ 1691,"Whited, Toni M",Is It Inefficient Investment That Causes the D...,NaN,2001.0


In [26]:
df.to_csv('new_tuned_references.csv',index=False,na_rep='NA')